In [4]:
from dotenv import load_dotenv
import os

In [5]:
load_dotenv() 
THE_MOVIE_DB_API_KEY = os.getenv("THE_MOVIE_DB_API_KEY")
THE_MOVIE_DB_ACCESS_TOKEN = os.getenv("THE_MOVIE_DB_ACCESS_TOKEN")
THE_MOVIE_DB_BASE_URL = os.getenv("THE_MOVIE_DB_BASE_URL")